In [28]:
import models.content_model

In [29]:
# load azure keys
import json
with open('../keys/azure.json') as json_file:
    azure_keys = json.load(json_file)
print(azure_keys.keys())


dict_keys(['stt_key1', 'stt_key2', 'openai_key1', 'openai_key2'])


In [30]:
# fetch files from url
import requests

def load_files_from_url(fileUrls):
    urls=fileUrls
    files=[]
    for url in urls:
        print("url------------------------\n",url)
        local_filename = url.split('/')[-1].split("%2F")[1].split("?")[0]
        local_filename='./data_dir/input_files/'+local_filename
        # NOTE the stream=True parameter below
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(local_filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        file_path=local_filename
        files.append(file_path)
    print("recieved file paths---------------------------\n",files)
    return files

# load_files_from_url([' https://firebasestorage.googleapis.com/v0/b/mh-pred-app.appspot.com/o/sdlc_cognizant%2Ffiles%2Fmail1.txt?alt=media&token=840a21e2-a35c-481f-9ebb-677854630d02'])

In [31]:
# processing text files
def proc_txt_data(files):
    txt_files=[f for f in files if '.txt' in f]
    txt_data=""
    for tf in txt_files:
        print(f"reading {tf} file----------------")
        with open(tf) as f:
            txt_data+='\n'.join(f.readlines())
    print("processed text data------------------------\n",txt_data)
    return txt_data

# txt_data=proc_txt_data(files)

In [32]:
# extract data from azure speech to text api : convert speech to text
import sys
import azure.cognitiveservices.speech as speechsdk

def proc_video_data(files):
    aud_files=[f for f in files if '.mp4' in f or '.wav' in f]
    
    aud_data=""
    for af in aud_files:
        speech_key, service_region = azure_keys['stt_key1'], "eastus"
        speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
        audio_config = speechsdk.audio.AudioConfig(filename=af)
        speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
        result = speech_recognizer.recognize_once()
        print(result.text)
        aud_data+="\n"+result.text
    return aud_data

# aud_data=proc_video_data(files)

In [33]:
content="""
At Cognizant, we have been on a quest to advance AI beyond existing techniques, by taking a more holistic,human-centric approach to learning and understanding. As Chief Technology Officer of Azure AI Services, 
I have been working with a team of amazing scientists and engineers to turn this quest into a reality. In my role, I enjoy a unique perspective in viewing the relationship among three attributes of human cognition:
monolingual text (X), audio or visual sensory signals, (Y) and multilingual (Z). At the intersection of all three,there’s magic—what we call XYZ-code as illustrated in Figure 1—a joint representation to create more powerful AI that can speak,
hear, see, and understand humans better. We believe XYZ-code will enable us to fulfill our long-term vision: cross-domain transfer learning, spanning modalities and languages. The goal is to have pre-trained models that can jointly learn representations to support a broad range of downstream AI tasks, much in the way humans do today. Over the past five years, we have achieved human performance on benchmarks in conversational speech recognition, machine translation, conversational question answering, machine reading comprehension, and image captioning. These five breakthroughs provided us with strong signals toward our more ambitious aspiration to produce a leap in AI capabilities, achieving multi-sensory and multilingual learning that is closer in line with how humans learn and understand. I believe the joint XYZ-code is a foundational component of this aspiration, if grounded with external knowledge sources in the downstream AI tasks.
"""

# content=txt_data+aud_data


In [34]:
# create summary from azure open ai api
def generate_summary(content):
    
    print("content to summarize----------------------\n",content.__str__())
    # prompt=f""" 
    # Generate a summary of our team meeting, highlighting key action items, decisions made, and deadlines set in key points. 
    # #meeting transcript start:
    # {content}
    # #meeting transcript end
    # """

    prompt=f""" 
    Generate a summary of our team meeting, highlighting in key points:
    1)Introduction
    2)Requirements Overview
    3) In scope
    4) out of scope
    5) assumptions
    6) Notes
     
    #meeting transcript start:
    {content}
    #meeting transcript end
    """
    from openai import AzureOpenAI
    # from azure.identity import DefaultAzureCredential, get_bearer_token_provider
    deployment = "red-gpt"
    endpoint="https://genai-warriors-openai-test.openai.azure.com/"
    # token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        # azure_ad_token_provider=token_provider,
        api_version="2024-02-01",
        api_key=azure_keys['openai_key1']
    )
    completion = client.completions.create(
        model=deployment,
        prompt=prompt,
        temperature=0.3,
        max_tokens=350,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    response=completion
    print(response)
    cfr=response.choices[0].content_filter_results
    pfr=response.prompt_filter_results
    res=response.choices[0].text


    print("\n\nsummary--------------------------\n",res)
    print(cfr,'\n',pfr)
    # return response
    return res +'\n\nContent Filter results=\n'+pfr.__str__()
   

# response=generate_summary(content)

In [35]:
# listener function
from models.content_model import ContentModel
# %pip install firebase_admin
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import datetime
import json
import threading
import time

from pandas import Timestamp



def listen_msgs():
    coll_name='sdlc_cognizant'
    user_uid='sdlc_cognizant_user'
        
    # Use a service account.
    cred = credentials.Certificate(f'../keys/sa.json')
    if not firebase_admin._apps:
        app = firebase_admin.initialize_app(cred)

    db = firestore.client()
    
    # Create an Event for notifying main thread.
    callback_done = threading.Event()

    # Create a callback on_snapshot function to capture changes
    def on_snapshot(doc_snapshot, changes, read_time):
        for doc in doc_snapshot:
            # print("doc-------------------------------\n",doc_snapshot)
            print(f"Received document snapshot: {doc.id}")
            print(doc.to_dict())
            content=ContentModel.from_dict(doc.to_dict())
            # complaint.complaintText=complaint.complaintText.toString()
            # data=doc.to_dict()
            # if content.audioURL!='':
            #     content.complaintText=convert_stt(complaint=content)
            # if content.lang!='english':
            #     content.complaintText=translate(complaint=content)

            files=load_files_from_url(content.fileUrls)
            txt_data=proc_txt_data(files=files)
            vid_data=proc_video_data(files=files)
            summary=generate_summary(txt_data+vid_data)
            # pred_res={'output':"model response"}

            content.senderId='backend@red'
            content.output=summary
            # data['priority']=pred_res['output'].split(':')[-1]
            content.ots=datetime.datetime.utcnow()
            print(f'sending response: '+content.output)
            # doc_id=str(round(time.time() * 1000))
            doc_id=content.id
            db.collection(coll_name).document(user_uid).collection('pContents').document(doc_id).set(content.__dict__)
            print(f"----------sent----------------with id: {doc_id} ")

        callback_done.set()

    doc_ref = db.collection(coll_name).document(user_uid).collection('rContents').document("content")

    # Watch the document
    doc_watch = doc_ref.on_snapshot(on_snapshot)
    print("listening for messages...",)
    
    

    while True:
        print('', end='', flush=True)
        time.sleep(1)



In [36]:
# calling the listener function
try:
    listen_msgs()
except Exception as e:
    print(f'error occured:\n\n\n {e}')  


listening for messages...
Received document snapshot: content
{'ots': None, 'senderId': 'user@red', 'fileUrls': ['https://firebasestorage.googleapis.com/v0/b/mh-pred-app.appspot.com/o/sdlc_cognizant%2Fmail1.txt?alt=media&token=81f65ed5-22bd-4343-a833-842cabe42188'], 'id': '1715608659838', 'output': '', 'timestamp': DatetimeWithNanoseconds(2024, 5, 13, 13, 57, 39, 838000, tzinfo=datetime.timezone.utc)}
url------------------------
 https://firebasestorage.googleapis.com/v0/b/mh-pred-app.appspot.com/o/sdlc_cognizant%2Fmail1.txt?alt=media&token=81f65ed5-22bd-4343-a833-842cabe42188
recieved file paths---------------------------
 ['./data_dir/input_files/mail1.txt']
reading ./data_dir/input_files/mail1.txt file----------------
processed text data------------------------
 At Cognizant, we have been on a quest to advance AI beyond existing techniques, by taking a more holistic,human-centric approach to learning and understanding. As Chief Technology Officer of Azure AI Services, 

I have been 

KeyboardInterrupt: 